# Battle of the Neighborhoods

## IBM Coursera Capstone Project

In [129]:
import numpy as np
import pandas as panda
import requests
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as panda
import folium
from matplotlib import pyplot as plot
from itertools import chain
from sklearn.cluster import KMeans


In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [3]:
wiki_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_data_as_string = requests.get(wiki_link).text

In [4]:
soup = BeautifulSoup(wiki_data_as_string,'lxml')

We inspected the wikipedia page detailing Toronto neighborhoods using chrome developer tools. We found out that we are lookign for a table with the class details as 
#### class="wikitable sortable jquery-tablesorter"

In [5]:
toronto_table = soup.find('table',{'class':'wikitable sortable'})

In [6]:
toronto_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [7]:
toronto_neighborhood = defaultdict(list)

for row in toronto_table.findAll("tr"):
    cells = row.findAll(["td"])
    if cells:
        cell_text = [i.text.strip() for i in cells]
        toronto_neighborhood['postcode'].append(cell_text[0])
        toronto_neighborhood['borough'].append(cell_text[1])
        toronto_neighborhood['neighborhood'].append(cell_text[1])
        
    

In [8]:
toronto_neighborhood

defaultdict(list,
            {'postcode': ['M1A',
              'M2A',
              'M3A',
              'M4A',
              'M5A',
              'M5A',
              'M6A',
              'M6A',
              'M7A',
              'M8A',
              'M9A',
              'M1B',
              'M1B',
              'M2B',
              'M3B',
              'M4B',
              'M4B',
              'M5B',
              'M5B',
              'M6B',
              'M7B',
              'M8B',
              'M9B',
              'M9B',
              'M9B',
              'M9B',
              'M9B',
              'M1C',
              'M1C',
              'M1C',
              'M2C',
              'M3C',
              'M3C',
              'M4C',
              'M5C',
              'M6C',
              'M7C',
              'M8C',
              'M9C',
              'M9C',
              'M9C',
              'M9C',
              'M1E',
              'M1E',
              'M1E',
              'M2E',
    

In [9]:
toronto_neighborhood= panda.DataFrame(toronto_neighborhood)
toronto_neighborhood.head()

,postcode,borough,neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,North York
3,M4A,North York,North York
4,M5A,Downtown Toronto,Downtown Toronto


In order to add the latitude and longitude co-ordinates with already webscraped data that we have collected, we are going to use the geospatial co ordinates file provided.

1. Load the excel using pandas
2. Join on postal code with already created table

In [10]:
geospatial_coordinates  = 'Geospatial_Coordinates.csv'
geo_data = panda.read_csv(geospatial_coordinates)
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
geo_data.rename(columns={'Postal Code':'postcode'}, inplace=True)
geo_data.head(1)

,postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353


In [12]:
toronto_neighborhood_with_coordinates= panda.merge(toronto_neighborhood,geo_data, how='left', on='postcode')
toronto_neighborhood_with_coordinates.shape, toronto_neighborhood.shape

((289, 5), (289, 3))

In [13]:
toronto_neighborhood_with_coordinates.head()

,postcode,borough,neighborhood,Latitude,Longitude
0,M1A,Not assigned,Not assigned,NaN,NaN
1,M2A,Not assigned,Not assigned,NaN,NaN
2,M3A,North York,North York,43.753259,-79.329656
3,M4A,North York,North York,43.725882,-79.315572
4,M5A,Downtown Toronto,Downtown Toronto,43.654260,-79.360636


In [14]:
toronto_neighborhood_with_coordinates.columns  = ['PostalCode','Borough','Neighborhood','Latitude','Longitude']

In [15]:
toronto_neighborhood_with_coordinates.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A,Not assigned,Not assigned,NaN,NaN
1,M2A,Not assigned,Not assigned,NaN,NaN
2,M3A,North York,North York,43.753259,-79.329656
3,M4A,North York,North York,43.725882,-79.315572
4,M5A,Downtown Toronto,Downtown Toronto,43.654260,-79.360636


Lets focus on Toronto boroughs for now

In [16]:
data = toronto_neighborhood_with_coordinates[toronto_neighborhood_with_coordinates.Borough.str.contains('Toronto')]
data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
4,M5A,Downtown Toronto,Downtown Toronto,43.654260,-79.360636
5,M5A,Downtown Toronto,Downtown Toronto,43.654260,-79.360636
17,M5B,Downtown Toronto,Downtown Toronto,43.657162,-79.378937
18,M5B,Downtown Toronto,Downtown Toronto,43.657162,-79.378937
34,M5C,Downtown Toronto,Downtown Toronto,43.651494,-79.375418


In [17]:
##lets check for null /empty /NAN values

data.isnull().any()

PostalCode      False
Borough         False
Neighborhood    False
Latitude        False
Longitude       False
dtype: bool

Lets plot the locations in a world map for visualization

In [29]:
# create map of Toronto using latitude and longitude values
map_newyork = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
#         parse_html=False
    ).add_to(map_newyork)  
    
map_newyork

Lets fetch the details of top venues around each location using our foursquare api

In [36]:
foursquare_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={0}&client_secret={1}&v={2}&ll={3},{4}&radius={5}&limit={6}'

CLIENT_ID = '4MXIJYYVXBKI4OR3XFDZH0IP1IWKFDUKEU3YYINP23D410XM' # your Foursquare ID
CLIENT_SECRET = 'OVVCVA5XAKRYE4VQWYZ0MXLRAA1ONJOE4DUBRNGEBALXSOSH' # your Foursquare Secret



In [57]:
def getNearbyVenues(names, latitudes, longitudes,url = foursquare_url, radius=500, limit =100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
        # create the API request URL
        url = foursquare_url.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            '20180605', #version 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

   
    return venues_list

In [46]:
venues_list = getNearbyVenues(
    names = data['Neighborhood'],
    latitudes = data['Latitude'],
    longitudes = data['Longitude']
)
venues_list

[[('Downtown Toronto',
   43.6542599,
   -79.3606359,
   'Roselle Desserts',
   43.653446723052674,
   -79.3620167174383,
   'Bakery'),
  ('Downtown Toronto',
   43.6542599,
   -79.3606359,
   'Tandem Coffee',
   43.65355870959944,
   -79.36180945913513,
   'Coffee Shop'),
  ('Downtown Toronto',
   43.6542599,
   -79.3606359,
   'Toronto Cooper Koo Family Cherry St YMCA Centre',
   43.65319052672638,
   -79.35794700053884,
   'Gym / Fitness Center'),
  ('Downtown Toronto',
   43.6542599,
   -79.3606359,
   'Morning Glory Cafe',
   43.653946942635294,
   -79.36114884214422,
   'Breakfast Spot'),
  ('Downtown Toronto',
   43.6542599,
   -79.3606359,
   'Body Blitz Spa East',
   43.65473505045365,
   -79.35987433132891,
   'Spa'),
  ('Downtown Toronto',
   43.6542599,
   -79.3606359,
   'Impact Kitchen',
   43.65636850543279,
   -79.35697968750694,
   'Restaurant'),
  ('Downtown Toronto',
   43.6542599,
   -79.3606359,
   'Figs Breakfast & Lunch',
   43.65567455427388,
   -79.364503289249

In [56]:
nearby_data = panda.DataFrame(list(chain.from_iterable(venues_list)))
nearby_data.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
print('Total venues details downloaded:',nearby_data.shape)
nearby_data.head()

Total venues details downloaded: (3286, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Toronto,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Downtown Toronto,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Downtown Toronto,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Downtown Toronto,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,Downtown Toronto,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [59]:
nearby_data.groupby(['Neighborhood']).head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Toronto,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Downtown Toronto,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Downtown Toronto,43.654260,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Downtown Toronto,43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,Downtown Toronto,43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
396,East Toronto,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
397,East Toronto,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
398,East Toronto,43.676357,-79.293031,Ava's Appletree,43.680100,-79.290700,Baby Store
399,East Toronto,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
852,West Toronto,43.669005,-79.442259,The Greater Good Bar,43.669409,-79.439267,Bar


#### Observations:

#### 1. Venue category is a categorical field. We can categorize the same using panda get dummies method
#### 2. Altogether there are 3 unique groups. During our analysis we can drop the field Venue..it provides no special details specific to our analysis

P.S In case we wanted to drill down further, eg analyze neighborhoods based on quality/reviews of each venue or say cluster neighborhoods based on quality of food, we would require venue to download review set from foursquare api


In [115]:
nearby_encoded = panda.get_dummies(nearby_data[['Venue Category']], prefix="", prefix_sep="")
nearby_encoded.head(2)

,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [116]:
nearby_encoded['Neighborhood']=nearby_data['Neighborhood']
columns = ['Neighborhood']
columns.extend([i for i in nearby_encoded.columns.tolist() if i not in ['Neighborhood']])


In [117]:

nearby_encoded = nearyby_encoded[columns]
nearby_encoded.tail()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
3281,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3282,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3283,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3284,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3285,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [125]:
data_for_clustering = nearby_encoded.groupby(['Neighborhood']).mean().reset_index()
data_for_clustering.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034632,...,0.017316,0.000000,0.012987,0.000000,0.000000,0.021645,0.000000,0.000000,0.000000,0.004329
1,Downtown Toronto,0.000401,0.000401,0.002807,0.002807,0.002807,0.005613,0.005613,0.005613,0.018043,...,0.000401,0.002807,0.010425,0.002005,0.000401,0.007217,0.007217,0.000401,0.001604,0.001604
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022099,...,0.011050,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022099
3,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005263,...,0.000000,0.000000,0.010526,0.000000,0.000000,0.010526,0.005263,0.000000,0.000000,0.005263


In [127]:
data_for_clustering.drop(['Neighborhood'], axis =1,inplace=True)
data_for_clustering.head()

,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034632,0.000000,...,0.017316,0.000000,0.012987,0.000000,0.000000,0.021645,0.000000,0.000000,0.000000,0.004329
1,0.000401,0.000401,0.002807,0.002807,0.002807,0.005613,0.005613,0.005613,0.018043,0.001203,...,0.000401,0.002807,0.010425,0.002005,0.000401,0.007217,0.007217,0.000401,0.001604,0.001604
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022099,0.000000,...,0.011050,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022099
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005263,0.005263,...,0.000000,0.000000,0.010526,0.000000,0.000000,0.010526,0.005263,0.000000,0.000000,0.005263


In [137]:
data_for_clustering.shape

(4, 238)

In [136]:
## Because all our data is mean,we can simply take the values as is without putting it through scaling/normalization
# set number of clusters
kclusters = 4

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(data_for_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 3, 2, 1])